This is a continuation of a [notebook](./differentials_rbf_NANs.ipynb) involving a case where the computation of differentials goes awry. In this notebook, this numerical error does not show up. We first show how differentials can be computed and then point out the implementation differences between this notebook and the previous.

To this end, we make use of the `modules` folder, which contains the definitions of the classes and modules that will not case such numerical errors.

In [1]:
import torch
import sys
sys.path.append('../../modules/')
from nn_rbf import RBF_Fix_All
from notable_kernels import gaussian_kernel

eps = 5.

In [2]:
torch.autograd.set_detect_anomaly(True)

We create an instance of the RBF interpolator equipped with the Gaussian kernel and $(0,0)$ as its center:

In [3]:
phi_rbf = RBF_Fix_All(centers=torch.zeros((1,2)), rbf_kernel=gaussian_kernel(eps))

with torch.no_grad(): # just to avoid the constant factor
    phi_rbf.output_layer.weight = torch.nn.Parameter(torch.Tensor([1]).reshape(1,1))

In [5]:
def phi_rbf_compare(x: torch.Tensor, y: torch.Tensor):
    return torch.exp(-eps**2*(x ** 2 + y ** 2))

We create the same grid as in the previous notebook:

In [6]:
xy = torch.cartesian_prod(*[torch.linspace(0, 1, 3,requires_grad=True), 
                            torch.linspace(0, 1, 3, requires_grad=True)])
x, y = xy[:, 0], xy[:, 1]
print(x)
print(y)

tensor([0.0000, 0.0000, 0.0000, 0.5000, 0.5000, 0.5000, 1.0000, 1.0000, 1.0000],
       grad_fn=<SelectBackward0>)
tensor([0.0000, 0.5000, 1.0000, 0.0000, 0.5000, 1.0000, 0.0000, 0.5000, 1.0000],
       grad_fn=<SelectBackward0>)


We verify the correctness of our module with regards to the explicitly-defined function:

In [8]:
phi_rbf_compare(x, y).reshape(-1,1) - phi_rbf(xy)

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SubBackward0>)

We now evaluate the derivatives along $x$ and $y$ on the points prescribed by the grid:

In [9]:
u = phi_rbf(torch.cat((x.unsqueeze(1),y.unsqueeze(1)), dim=1))
grad_x, grad_y = torch.autograd.grad(u, (x,y), 
                                     grad_outputs=torch.ones_like(u), 
                                     create_graph=True)

We compare the obtained results with the result of computing the differential by hand and then evaluating it, seeing that they are equal:

In [10]:
print(grad_x - -2*eps**2*x*torch.exp(-eps**2*(x**2+y**2)))
print(grad_y - -2*eps**2*y*torch.exp(-eps**2*(x**2+y**2)))

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SubBackward0>)
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SubBackward0>)


We do likewise, computing the second derivatives in $x$ and $y$:

In [11]:
u_xx = torch.autograd.grad(grad_x, x, grad_outputs=torch.ones_like(grad_x), create_graph=True, retain_graph=True)[0]
u_yy = torch.autograd.grad(grad_y, y, grad_outputs=torch.ones_like(grad_y), create_graph=True, retain_graph=True)[0]

We compare the results yielded by PyTorch with regards to the derivative computed by hand:

In [12]:
u_xx - 2*eps**2*(-1+2*eps**2*x**2)*torch.exp(-eps**2*(x**2+y**2))

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1921e-07, 0.0000e+00, 0.0000e+00,
        3.5527e-15, 0.0000e+00, 0.0000e+00], grad_fn=<SubBackward0>)

In [13]:
u_yy - 2*eps**2*(-1+2*eps**2*y**2)*torch.exp(-eps**2*(x**2+y**2))

tensor([0.0000e+00, 1.1921e-07, 3.5527e-15, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00], grad_fn=<SubBackward0>)

There are some numerical discrepancies. However, they are not very significant for most applications (being at the seventh decimal place in the worst case observed).

------

What are we doing differently, so that NANs do not show up? We simply compare the definition provided in the previous notebook with the one featured in the `modules` folder:

In [ ]:
#### This implementation causes NANs when differentiating

def compute_radii(x: torch.Tensor, centers: torch.Tensor) -> torch.Tensor:
    x = x.unsqueeze(1)  # Shape: (batch_size, 1, d)
    centers = centers.unsqueeze(0)  # Shape: (1, num_centers, d)
    
    squared_distances = torch.sum((x - centers)**2, dim=2)  # Shape: (batch_size, num_centers)
    distances = torch.sqrt(squared_distances)
    
    return distances

def gaussian_kernel(eps: float | torch.Tensor):
    def fn(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        radii = compute_radii(x, y)
        return torch.exp(-(eps * radii) ** 2)
    return fn

In [ ]:
#### This implementation won't cause NANs when differentiating

def compute_radii_squared(x: torch.Tensor, centers: torch.Tensor) -> torch.Tensor:
    x = x.unsqueeze(1)  # Shape: (batch_size, 1, d)
    centers = centers.unsqueeze(0)  # Shape: (1, num_centers, d)
    
    return torch.sum((x - centers)**2, dim=2)  # Shape: (batch_size, num_centers)


def gaussian_kernel(eps: float | torch.Tensor):
    def fn(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        radii_sq = compute_radii_squared(x, y)
        return torch.exp(-eps ** 2 * radii_sq)
    return fn

The error message in the previous notebook details that NANs appear differentiating, concretely in the differential of the square root. This function is not differentiable at $x=0$. In the case the two inputs coincide for function `compute_radii()`, this error will show up.

Because the Gaussian kernel uses the square of the radius anyway, instead of computing the radius we just do not take its square root, and account for this change in the kernel.